In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
import pandas as pd
os.chdir('/path of gene expression/')
from scipy import stats
import math

In [ ]:
#导入数据
data = pd.read_csv("gene.csv",header = 0,index_col = 0)
data.head()

In [ ]:
# 查看数据维度
# Check the dimension of the loaded data (rows & columns)
print(data.shape)
# Number of rows
number_of_genes = len(data.index)
print(number_of_genes)

In [ ]:
# Boxplot of each microarray
plt.show(data2.plot(kind = 'box', title = 'gene Boxplot', rot = 90))

In [ ]:
# Density
plt.show(data.plot(kind = 'density', title = 'gene Density'))

In [ ]:
#差异分析
# The mean of expression of the tumor samples for each gene (row)
wt = data.loc[:, 'R01-003':'R01-146'].mean(axis = 1)
wt.head()

In [ ]:
# The mean of expression of the normal samples for each gene (row)
ko = data.loc[:,'lung3' : 'lung1'].mean(axis = 1)
ko.head()

In [ ]:
# 查看基因差异的差异倍数fold分布
fold = wt / ko
foldchange = np.log2(fold)
#print(foldchange)
# Histogram of the fold-change
plt.hist(foldchange)
plt.title("Histogram of fold-change")
plt.show()

In [ ]:
#查看基因差异的P值分布
from scipy import stats

pvalue = []
for i in range(0, number_of_genes):
    ttest = stats.ttest_ind(data.iloc[i,0:2], data.iloc[i,3:])
    pvalue.append(ttest[1])

# Histogram of the p-values
plt.hist(-np.log10(pvalue))
#print(-np.log10(pvalue))
plt.title("Histogram of p-value")
plt.show()

In [ ]:
#火山图
myarray = np.asarray(pvalue)
result = pd.DataFrame({'pvalue':myarray,'FoldChange':foldchange})
result['log(pvalue)'] = -np.log10(result['pvalue'])
result['sig'] = 'Normal'
result['size']  =np.abs(result['FoldChange'])/10

result.loc[(result.FoldChange> 1.5 )&(result.pvalue < 0.05),'sig'] = 'Over'
result.loc[(result.FoldChange< -1.5 )&(result.pvalue < 0.05),'sig'] = 'Low'
plt.figure(figsize=(6,5))
ax = sns.scatterplot(x="FoldChange", y="log(pvalue)",
                     hue_order = ('Low','Normal','Over'), hue='sig',
                     palette=("#377EB8","grey","#E41A1C"), s=15,
                     data=result)
ax.axis([-6,6,0,10])
ax.axvline(1.5, 0,10,linestyle="--",alpha=0.55,color='grey')
ax.axvline(-1.5, 0,10,linestyle="--",alpha=0.55,color='grey')
ax.axhline(1.3, -1,6,linestyle="--", alpha=0.55,color='grey')
#ax.text(-6,10.1,"Low Expression <--")
#ax.text(2.9,10.1,"--> Over Expression")
ax.text(-1.63,-0.527,"-1.5")
ax.text(1.23,-0.527,"1.5")
ax.text(-6.65,1.18,"1.3")
ax.text(-7.1,4.3,"10",fontweight='bold',fontsize=6.5,rotation=90)
ax.text(-0.2,-1.1,"2",fontweight='bold',fontsize=6.5)
#ax.annotate('HOPX',
 #           xy=(-1.6, 3.6922), xycoords='data',
  #          xytext=(0.5,0.5), textcoords='axes fraction',
   #         arrowprops=dict(facecolor='black', shrink=0.05),
    #        horizontalalignment='right', verticalalignment='top')
ax.set_ylabel('-log  (p-value)',fontweight='bold',fontsize=11)
ax.set_xlabel('log (FC)',fontweight='bold',fontsize=11)
#ax.set_title('Volcano Plot of DEA',y=1,fontweight='bold',fontsize=15)
ax.figure.savefig('/Users/armandayuu/Desktop/dea.png',dpi=800,bbox_inches='tight')

In [ ]:
#筛选差异基因
fold_cutoff = -1.5
pvalue_cutoff = 0.05

filtered_ids = []
for i in range(0, number_of_genes):
    if (abs(fold[i]) >= fold_cutoff) and (pvalue[i] <= pvalue_cutoff):
        filtered_ids.append(i)
        
filtered = data.iloc[filtered_ids,:]
print("Number of DE genes: ")
print(len(filtered.index))

filtered.head(60)
